In [1]:
%load_ext Cython

#pseudocode
'''
define a lattice with random configuration
each lattice site has a plane of rotations that alternates**

'''

'\ndefine a lattice with random configuration\neach lattice site has a plane of rotations that alternates**\n\n'

In [2]:
%%cython -a

from __future__ import division

import numpy as np
import random as random
cimport numpy as np
cimport cython

from numpy cimport abs
from scipy import constants
from libcpp cimport bool
from libc.math cimport exp
from libc.stdlib cimport rand

cdef extern from "limits.h":
    int RAND_MAX


def random_lattice(int N, int M):
    cdef np.float64_t[:] theta =  np.random.uniform(0,2*np.pi, N*M)

    L = np.array(np.zeros((N,M), dtype=np.ndarray))
    u1 = np.array([1/np.sqrt(2),1/np.sqrt(2),0])
    u2 = np.array([-1/np.sqrt(2),1/np.sqrt(2),0])
    
    a = 0
    b = 0
    temp = False
    for i in range(N):
        for j in range(M):
            if a % 2 == 0:
                R1 = rotation(theta, i, True)
                L[i,j] = R1.dot(u2)

            else:
                R2 = rotation(theta, i, False)
                L[i,j] = R2.dot(u1)
                
            if (a+1+b) % N == 0 and a != 0 and temp == False:
                temp = True
                b = b + 1
            else:
                a = a + 1
                temp = False
  
    return L

@cython.boundscheck(False)
@cython.wraparound(False)
cdef rotation(np.float64_t[:] theta, int i, np.npy_bool c):
    
    cdef np.float64_t[:] C = np.cos(theta)
    cdef np.float64_t[:] S = np.sin(theta)
    cdef np.float64_t[:] t = (1-np.cos(theta))
    cdef np.float64_t[:] u1 = np.array([1/np.sqrt(2),1/np.sqrt(2),0])
    cdef np.float64_t[:] u2 = np.array([-1/np.sqrt(2),1/np.sqrt(2),0])
   
    if c == True:
        R1 = np.array([[C[i]+np.power(u1[0],2)*t[i], u1[0]*u1[1]*t[i]-u1[2]*S[i], u1[0]*u1[2]*t[i]+u1[1]*S[i]],
      [u1[1]*u1[0]*t[i]+u1[2]*S[i], C[i]+np.power(u1[1],2)*t[i], u1[1]*u1[2]*t[i]-u1[0]*S[i]],
      [u1[2]*u1[0]*t[i]-u1[1]*S[i], u1[2]*u1[1]*t[i]+u1[0]*S[i], C[i]+np.power(u1[2],2)*t[i]]])

        return R1

    else:
        R2 = np.array([[C[i]+np.power(u2[0],2)*t[i], u2[0]*u2[1]*t[i]-u2[2]*S[i], u2[0]*u2[2]*t[i]+u2[1]*S[i]],
      [u2[1]*u2[0]*t[i]+u2[2]*S[i], C[i]+np.power(u2[1],2)*t[i], u2[1]*u2[2]*t[i]-u2[0]*S[i]],
      [u2[2]*u2[0]*t[i]-u2[1]*S[i], u2[2]*u2[1]*t[i]+u2[0]*S[i], C[i]+np.power(u2[2],2)*t[i]]])

        return R2

@cython.boundscheck(False)
@cython.wraparound(False)
cdef unit_vector(vector):
    return vector / np.linalg.norm(vector)

@cython.boundscheck(False)
@cython.wraparound(False)
cdef angle_between(v1, v2):
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.dot(v1_u, v2_u))
    
def cy_aa_step(L, beta):
    N = L.shape[0]
    M = L.shape[1]
    i = 0
    k = 0
    temp = False
    for n in range(N):
        for m in range(M):
            cy_aa_update(L, n, m, beta, i)
            if (i+1+k) % N == 0 and i != 0 and temp == False:
                temp = True
                k = k + 1
            else:
                i = i + 1
                temp = False
    return L


@cython.boundscheck(False)
@cython.wraparound(False)
cdef cy_aa_update(np.ndarray[:,:] L, int n, int m, float beta, int i):
    
    cdef int N = L.shape[0]
    cdef int M = L.shape[1]
    cdef int[:] r1 = np.array([0,-1,0])
    cdef int[:] r2 = np.array([0,1,0])
    cdef int[:] r3 = np.array([1,0,0])
    cdef int[:] r4 = np.array([-1,0,0])
    
    cdef int[:] r2_2 = np.array([1,1,0])
    cdef int[:] r1_2 = np.array([-1,1,0])
    cdef int[:] r3_2 = np.array([1,-1,0])
    cdef int[:] r4_2 = np.array([-1,-1,0])
    
    cdef np.float64_t[:] theta = np.random.uniform(0,2*np.pi,1)
    
    if (i)%2 == 0:
        c = True
    else:
        c = False
        
    R = rotation(theta, 0, c)
    r = R.dot(L[n,m])
    
#initial
    #nearest neighbors
    cdef float H1_i = -1*(3*(L[n,m].dot(r1))*(L[(n + 1)%N,m].dot(r1))-(L[n,m].dot(L[(n + 1)%N, m])))
    cdef float H2_i = -1*(3*(L[n,m].dot(r2))*(L[(n - 1)%N, m].dot(r2))-(L[n,m].dot(L[(n - 1)%N, m])))
    cdef float H3_i = -1*(3*(L[n,m].dot(r3))*(L[n, (m + 1)%M].dot(r3))-(L[n,m].dot(L[n, (m + 1)%M])))
    cdef float H4_i = -1*(3*(L[n,m].dot(r4))*(L[n, (m - 1)%M].dot(r4))-(L[n,m].dot(L[n, (m - 1)%M])))
    #2nd nearest neighbors
    cdef float H1_2i = -(1/np.sqrt(2)**3)*(3*(L[n,m].dot(r1_2))*(L[(n + 1)%N,(m + 1)%M].dot(r1_2))-(L[n,m].dot(L[(n + 1)%N,(m + 1)%M])))
    cdef float H2_2i = -(1/np.sqrt(2)**3)*(3*(L[n,m].dot(r2_2))*(L[(n + 1)%N, (m - 1)%M].dot(r2_2))-(L[n,m].dot(L[(n + 1)%N, (m - 1)%M])))
    cdef float H3_2i = -(1/np.sqrt(2)**3)*(3*(L[n,m].dot(r3_2))*(L[(n - 1)%N, (m + 1)%M].dot(r3_2))-(L[n,m].dot(L[(n - 1)%N, (m + 1)%M])))
    cdef float H4_2i = -(1/np.sqrt(2)**3)*(3*(L[n,m].dot(r4_2))*(L[(n - 1)%N, (m - 1)%M].dot(r4_2))-(L[n,m].dot(L[(n - 1)%N, (m - 1)%M])))

#proposed
    #nearest neighbors
    cdef float H1_f = -1*(3*(r.dot(r1))*(L[(n + 1)%N,m].dot(r1))-(r.dot(L[(n + 1)%N, m])))
    cdef float H2_f = -1*(3*(r.dot(r2))*(L[(n - 1)%N, m].dot(r2))-(r.dot(L[(n - 1)%N, m])))
    cdef float H3_f = -1*(3*(r.dot(r3))*(L[n, (m + 1)%M].dot(r3))-(r.dot(L[n, (m + 1)%M])))
    cdef float H4_f = -1*(3*(r.dot(r4))*(L[n, (m - 1)%M].dot(r4))-(r.dot(L[n, (m - 1)%M])))
    #2nd nearest neighbors
    cdef float H1_2f = -(1/np.sqrt(2)**3)*(3*(r.dot(r1_2))*(L[(n + 1)%N,(m + 1)%M].dot(r1_2))-(r.dot(L[(n + 1)%N,(m + 1)%M])))
    cdef float H2_2f = -(1/np.sqrt(2)**3)*(3*(r.dot(r2_2))*(L[(n + 1)%N, (m - 1)%M].dot(r2_2))-(r.dot(L[(n + 1)%N, (m - 1)%M])))
    cdef float H3_2f = -(1/np.sqrt(2)**3)*(3*(r.dot(r3_2))*(L[(n - 1)%N, (m + 1)%M].dot(r3_2))-(r.dot(L[(n - 1)%N, (m + 1)%M])))
    cdef float H4_2f = -(1/np.sqrt(2)**3)*(3*(r.dot(r4_2))*(L[(n - 1)%N, (m - 1)%M].dot(r4_2))-(r.dot(L[(n - 1)%N, (m - 1)%M])))
    
    cdef np.float64_t[:] Lp = np.array(L[n,m])
    cdef np.float64_t[:] rp = np.array(r)
    Lp[2] = 0.0
    rp[2] = 0.0
    
    cdef np.float64_t[:] tl = np.array([angle_between(L[n,m], Lp)])
    cdef np.float64_t[:] tr = np.array([angle_between(r, rp)])
    cdef float ramp = 23.77165877


    cdef float H_i = H1_i + H2_i + H3_i + H4_i + H1_2i + H2_2i + H3_2i + H4_2i + ramp*np.sin(tl)
    cdef float H_f = H1_f+ H2_f + H3_f + H4_f + H1_2f+ H2_2f + H3_2f + H4_2f + ramp* np.sin(tr)
    cdef float H = H_f - H_i

    if H <= 0:
        L[n, m] = r 

    elif rand() < RAND_MAX * exp(-beta * H):
        L[n, m] = r 

C:\Users\Andrew\Anaconda2\lib\site-packages\Cython\Compiler\Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: C:\Users\Andrew\.ipython\cython\_cython_magic_7b4e3451273c93d49ec9d73c6a72e906.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [7]:
from mayavi import mlab
import numpy as np
%gui qt

#msplt = plt.mlab_source
@mlab.animate(delay=1000)
def anim(L):
    while True:
        L = cy_aa_step(L, .3)
        A = []
        for i in range(N):
            for j in range(N):
                A.append(L[j][i].tolist())
        U, V, W = zip(*np.stack(A))
        plt.mlab_source.set(x=X,y=Y,z=Z,u=U,v=V,w=W)  
        yield

        

N = 4
L = random_lattice(N,N)
A = []

for i in range(N):
    for j in range(N):
        A.append(L[j][i].tolist())
        
X = [b for b in range(N) for x in range(N)]
Y = [x for b in range(N) for x in range(N)]
Z = [0]*N*N
X = tuple(X)
Y = tuple(Y)
Z = tuple(Z)
f = mlab.figure()

U, V, W = zip(*np.stack(A))

plt = mlab.quiver3d(X,Y,Z,U,V,W)

anim(L)
mlab.show()

In [169]:
A = np.sum(L)
print A
np.linalg.norm(A, ord = 3)

[-2.02259748 -4.60953094  6.99601706]


7.655317562213603